In [1]:
import os
import sys

os.environ['SPARK_HOME']='/home/cloudera/spark230hadoop26'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.6-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
# spark = SparkSession.builder.appName('SparkTransformations') \
# .config('spark.warehouse.dir','/apps/hive/warehouse') \
# .enableHiveSupport().getOrCreate()
spark = SparkSession.builder.appName('SparkAccumulatorsAndBroadcastVariables') \
.enableHiveSupport().getOrCreate()

In [4]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'2.3.0'

In [5]:
sc = spark.sparkContext

In [6]:
sc.version

'2.3.0'

In [7]:
a = sc.accumulator(1)
a.value

1

In [8]:
rdd = sc.parallelize([1, 2, 3])
def f(x):
    global a
    a += x
rdd.foreach(f)
a.value

7

In [9]:
new_accum = sc.accumulator(0)
x = 1000000
nrdd = sc.parallelize(range(2, x))
def is_prime(x):
    import math
    if x == 1:
        return False
    if x == 2:
        return True
    return all(x % y != 0 for y in range(2, int(math.sqrt(x)) + 1))

def count_prime_accum(x):
    global new_accum
    if is_prime(x):
        new_accum += 1
        return x
    else:
        return 0
sum_of_primes_less_than_x = nrdd.filter(lambda x: count_prime_accum(x)).sum()
print(sum_of_primes_less_than_x)
print(new_accum.value)

37550402023
78498


In [10]:
from pyspark.accumulators import AccumulatorParam

class VectorAccumulatorParam(AccumulatorParam):
    def zero(self, value):
        return [0.0] * len(value)
    
    def addInPlace(self, val1, val2):
        for x in range(len(val1)):
            val1[x] += val2[x]
        return val1

va = sc.accumulator([1.0, 2.0, 3.0], VectorAccumulatorParam())
def g(x):
    global va
    va += [x] * 3
print(va.value)
rdd = sc.parallelize([10, 20, 30])
# [1, 2, 3] + [10, 10, 10] = [11, 12, 13] + [20, 20, 20] + [30, 30, 30] = [61, 62, 63]
rdd.foreach(g)
print(va.value)

[1.0, 2.0, 3.0]
[61.0, 62.0, 63.0]


In [79]:
nac = sc.accumulator(0)
def mappart(itera):
    global nac
    nac += 1
    yield list(map(lambda x: x * 2, itera))
ardd = sc.parallelize(range(10))
print(ardd.mapPartitions(mappart).collect())
print(nac.value)

[[0, 2], [4, 6, 8], [10, 12], [14, 16, 18]]
4


In [11]:
nmbr_lim_to_check = 1000000

In [12]:
prime_dict = {x: is_prime(x)  for x in range(nmbr_lim_to_check) }

In [13]:
prime_broadcast = sc.broadcast(prime_dict)

In [83]:
braccum = sc.accumulator(0)

def check_prime_using_brv(x):
    global braccum
    if prime_broadcast.value[x]:
        braccum += 1
        return x
    else:
        return 0

prime_numbers = sc.parallelize(range(nmbr_lim_to_check)).filter(lambda x: check_prime_using_brv(x))
print(prime_numbers.sum())
print(braccum.value)

37550402023
78499
